# Churn problem 

A bank is investigating a very high rate of customer leaving the bank. Here is a 10.000 records dataset to investigate and predict which of the customers are more likely to leave the bank soon.

In [28]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation
import seaborn as sns                               # Advanced data visualisation

%matplotlib inline

Read the file Churn_Modelling.csv.  
Show the first few records.

In [29]:
df = pd.read_csv('https://raw.githubusercontent.com/jdecorte/machinelearning/main/datasets/churn_modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


What are the dimensions of the datasets?

In [30]:
print("Shape of DF:", df.shape)

Shape of DF: (10000, 14)


Is this a skewed dataset?

In [31]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Drop the columns RowNumber, CustomerId and Surname

In [32]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

What are the different values for Geography?

In [33]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

Use one-hot encoding to replace Gender and Geography.

In [34]:
df = pd.get_dummies(df,prefix=['Geography'], columns = ['Geography'])
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [35]:
df = pd.get_dummies(df,prefix=['Gender'], columns = ['Gender'])
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


Make sure you understand this function. We will use is later on to evaluate the ANN.

In [36]:
def plot_history(history):
  plt.figure(figsize = (12,4))
  plt.subplot(1,2,1)

  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(history.epoch, np.array(history.history['accuracy']),'g-',
           label='Train accuracy')
  plt.plot(history.epoch, np.array(history.history['val_accuracy']),'r-',
           label = 'Validation accuracy')
  plt.legend()

  plt.subplot(1,2,2)
  plt.xlabel('Epoch')
  plt.ylabel('Loss minimised by model')
  plt.plot(history.epoch, np.array(history.history['loss']),'g-',
           label='Train loss')
  plt.plot(history.epoch, np.array(history.history['val_loss']),'r-',
           label = 'Validation loss')
  plt.legend()

What is X and what is y?

In [37]:
X = df.drop(["Exited"], axis = 1)
y = df['Exited']   

What is X_train, y_train, X_test, y_test?

In [38]:
from sklearn.model_selection import train_test_split    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  

What is X_train, y_train, X_val, y_val?

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)  

What are the dimensions of X_train, X_val, X_test, y_train, y_val, y_test?

In [40]:
print("Shape of x_train:", X_train.shape)
print("Shape of x_test:", X_test.shape)
print("Shape of x_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of y_val:", y_val.shape)

Shape of x_train: (6400, 13)
Shape of x_test: (2000, 13)
Shape of x_val: (1600, 13)
Shape of y_train: (6400,)
Shape of y_test: (2000,)
Shape of y_val: (1600,)


Standardize the data before providing it as training data.  
You should only use the training data to fit the scaler transformation
and use the same fitted object to transform the validation and test input
data.

In [41]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

* First create a small ANN model.  
* Add BatchNormalization
* Add Dropout
* Evaluate the results after each fine tuning.


In [65]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, activation="relu", input_dim = X_train_scaled.shape[1]))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1, activation="sigmoid"))

In [66]:
model.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [67]:
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=64,
                    validation_data=(X_val_scaled, y_val))

Epoch 1/20
100/100 [==============================] - 1s 4ms/step - loss: 0.7061 - accuracy: 0.6034 - val_loss: 0.5321 - val_accuracy: 0.8000
Epoch 2/20
100/100 [==============================] - 0s 3ms/step - loss: 0.5565 - accuracy: 0.7269 - val_loss: 0.4549 - val_accuracy: 0.8075
Epoch 3/20
100/100 [==============================] - 0s 3ms/step - loss: 0.5105 - accuracy: 0.7663 - val_loss: 0.4243 - val_accuracy: 0.8150
Epoch 4/20
100/100 [==============================] - 0s 3ms/step - loss: 0.4853 - accuracy: 0.7872 - val_loss: 0.4075 - val_accuracy: 0.8156
Epoch 5/20
100/100 [==============================] - 0s 2ms/step - loss: 0.4699 - accuracy: 0.7937 - val_loss: 0.3986 - val_accuracy: 0.8244
Epoch 6/20
100/100 [==============================] - 0s 2ms/step - loss: 0.4705 - accuracy: 0.7922 - val_loss: 0.3922 - val_accuracy: 0.8275
Epoch 7/20
100/100 [==============================] - 0s 2ms/step - loss: 0.4591 - accuracy: 0.8016 - val_loss: 0.3858 - val_accuracy: 0.8313
Epoch 